In [13]:
import torch # ML framework: It use tensor DS for math and calculus
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Hyperpara
block_size = 8
batch_size = 4 # How many blocks we process in ||
max_iters = 1000
eval_iters = 250
learning_rate = 3e-4
dropout = 0.2 # drop 20% random neurons

cuda


In [14]:
with open('data/magic_of_oz.txt', 'r', encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars) # How many unique chars in dataset
print(chars)
print(vocab_size)



['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
80


In [15]:
# tokenization -> encoding/deconding encoding convert each char to int

string_to_int = { char:index for index,char in enumerate(chars) }
int_to_string = { index:char for index,char in enumerate(chars) }
encode = lambda string: [string_to_int[char] for char in string]
decode = lambda int_list: ''.join([int_to_string[int] for int in int_list])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])



tensor([79,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 50, 33, 64, 64, 73, 71,
        72, 70, 53, 72, 61, 67, 66, 51,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1, 44, 60, 57,  1, 30, 53, 65, 67, 73, 71,  1, 39, 78,  1, 26,
        67, 67, 63, 71,  0,  0,  0, 43, 61, 66])


In [16]:
# Validation and training split
n = int(0.08 * len(data))
train_data = data[:n]
validation_data = data[n:]


In [29]:
def get_batch(split):
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(0, len(data) - block_size, (batch_size, ))
    # print(f"Random indices: {ix}")
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # Push data to GPU means its calc handled by GPU
    return x, y

x, y = get_batch('train')
print('Inputs:')
print(x)
print('Target:')
print(y)

Inputs:
tensor([[67, 73, 66, 72,  1, 37, 73, 66],
        [61, 63, 57,  1, 53,  1, 71, 53],
        [55, 53, 64, 64, 57, 56,  1, 60],
        [ 1,  1,  1,  1,  1, 27, 60, 61]], device='cuda:0')
Target:
tensor([[73, 66, 72,  1, 37, 73, 66, 55],
        [63, 57,  1, 53,  1, 71, 53, 73],
        [53, 64, 64, 57, 56,  1, 60, 61],
        [ 1,  1,  1,  1, 27, 60, 61, 55]], device='cuda:0')


In [18]:
# Bigram example
x = train_data[: block_size]
y = train_data[1: block_size+1]

for t in range(block_size):
    context = x[: t+1]
    target = y[t]
    print(f"When input is {context} target is {target}")

When input is tensor([79]) target is 1
When input is tensor([79,  1]) target is 1
When input is tensor([79,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1,  1]) target is 1


In [19]:
# decorator is used in PyTorch to temporarily disable gradient computation during the execution of the decorated function or code block.
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # Set the model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # Set the model back to training mode
    return out

In [20]:
# nn.Module is a fundamental building block used to create neural network architectures. It's a base class for all neural network modules.
# provides methods for various functionalities like managing parameters, defining layers, and executing forward propagation.
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:
            B, T, C = logits.shape # Batch, Time, Channels is vocab_size
            logits = logits.view(B*T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) # Logits are the raw scores or probabilities that a model assigns to each class
            
            logits = logits[:, -1, :] # On last time step became (B, C)
            probs = F.softmax(logits, dim=-1) # Apply softmax for probablities
            index_next = torch.multinomial(probs, num_samples=1) # Sample from distribution (B, 1)
            index = torch.cat((index, index_next), dim=1) # Append sample index to running index (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device) # torch.long in int64 integer
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars

'\nryWBhQGHVGd&\n1x1sZ?&WVl_BvHCE3T[e\ufeffOSZ0lD(,\'uCOcdCicUD,*O"HLTfbvz:VVjjtv3\ufeff!SC&SfU)\nrWBhbpGdpoTi0loD"C,&(K07QGC"j-2)6UR)o4ql\'i(RID;vaHV8VZAo_oAfJs87yBhBid943TGkEN7\ufeffLMVk?-xa2Q4v\nDMR-.8;Z56U&[vS&("JYQ9D:bjq]rr7nT[kNEbG2_T_r\ufeffvjeAH9P\'6-r47J,R9x\ufeffJx,3\ufeff5d;x(K&D-j7b:7zNl:0 "0;&Wla6]&yZH79[PaPR_R:U&dM?l&\'r\'iJr?Isnrrr8e*OSVd!hcd!\ntGj0j)M-tq1rrLFy]K&,?H]mf:20")\nrVEtR,YQt2TDlD0J\'5RU*ogkPh2MYBh:nB\';tu2O075e\'xy*(d?9!\',K[U)tsCv]kST5:UQR)9*;g\nfYu\nx:UKebZ-Cwc8GvHDWjRSsVxnT7. \n0yT[gH_PxZD8k("ief7SmY.3\ufeffvjmMZH.HI-.FV'

In [36]:
# Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f} val loss: {losses['val']:.4f}")

    # Eval loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # It reset prev gradients so they do not add with current gradient
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.8810 val loss: 3.0407
step: 250, train loss: 2.8867 val loss: 3.0459
step: 500, train loss: 2.8751 val loss: 3.0159
step: 750, train loss: 2.8560 val loss: 3.0105
3.040534734725952


In [22]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars

'\n!")]-;Fw6fb]K&_\ufefftahxc5shFxx8mQJTDSUR_FV\ufeffOStttF4BhF(8s["p"6.H.8(d2_\nN2Wu]\'gYyNon7hFM:3;rrhuZ&eFmL sj)9u.cS5gBJ!SZ!VME!(,\nBf&W99Pwe&(KNc5:;J5vEy*\ufefflI&Lv\ufeffZ,mg0huA)MrA!Dcc,Gv8AxCa&_!sxnS1\nW6otJ6G BhykSe5"5eYj&JsG_d4Du,&(8h\nV?DxC\ng]r21xzHVMKBWliGH82_5 BUPf[?*1yM!Ndxw:K0SZoz7bL TkcBh.4qsGaPfBlont7bk(2&\'nocj0j[\'sEj5P95R*GvpGb6-Lc5Uo,bLuRkgA6go:U,pJO?0.0hcgw\'DMEAZZ,OnQA4)6AGR)l5_6k?:UWq?eGIRNdy_\']ycJBpK0F4qZ\ufeffLaloS.?F4QH2H75,381nZ ,\'&_4zSVifzPBj\n0i W6v0jK"7HDWxCgV[ Il!l3rU0yH_J\'-n)MEQ*eY]K b_6fVk;g;hBhINc'